In [1]:
import sys
import os
import random

# Add the absolute path to the src directory 
os.chdir('../src')
src_path = os.path.abspath('../src/')
sys.path.insert(0, src_path)

from setup import *
seed = 123
import random
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

/opt/anaconda3/envs/dsc180/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All working on Pepties Func dataset

In [2]:
# load the data
mp.set_start_method('spawn', force=True)
# Load entire Peptides-func dataset
train_loader, test_loader, num_classes = load_peptides_func(parallel=True, subset_ratio=1.0)
print('Data done loading')
num_node_features = 9 

Loaded and split data
Data done loading


# GPS

In [3]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPSLongRange(
    num_node_features=num_node_features,
    hidden_channels=32,
    num_classes=num_classes,
    num_layers=5
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(200):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
        
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.5592, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 10 : Train Loss = 1.3358, Train Acc = 0.5801 , Test Acc = 0.5894
EPOCH 20 : Train Loss = 1.2685, Train Acc = 0.6043 , Test Acc = 0.6221
EPOCH 30 : Train Loss = 1.2119, Train Acc = 0.6234 , Test Acc = 0.6414
EPOCH 40 : Train Loss = 1.2068, Train Acc = 0.6182 , Test Acc = 0.6221
EPOCH 50 : Train Loss = 1.1773, Train Acc = 0.6108 , Test Acc = 0.6285
EPOCH 60 : Train Loss = 1.1706, Train Acc = 0.6181 , Test Acc = 0.6244
EPOCH 70 : Train Loss = 1.1469, Train Acc = 0.6370 , Test Acc = 0.6497
EPOCH 80 : Train Loss = 1.1208, Train Acc = 0.6428 , Test Acc = 0.6418
EPOCH 90 : Train Loss = 1.1159, Train Acc = 0.6482 , Test Acc = 0.6464
EPOCH 100 : Train Loss = 1.1106, Train Acc = 0.6460 , Test Acc = 0.6506
EPOCH 110 : Train Loss = 1.1034, Train Acc = 0.6556 , Test Acc = 0.6524
EPOCH 120 : Train Loss = 1.0986, Train Acc = 0.6535 , Test Acc = 0.6561
EPOCH 130 : Train Loss = 1.0903, Train Acc = 0.6574 , Tes

# GCN

In [3]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNGraph(
    num_node_features=num_node_features,
    hidden_channels=32,
    num_classes=num_classes,
    layers=3
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(150):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
        
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.6067, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 10 : Train Loss = 1.3901, Train Acc = 0.5430 , Test Acc = 0.5660
EPOCH 20 : Train Loss = 1.3710, Train Acc = 0.5499 , Test Acc = 0.5710
EPOCH 30 : Train Loss = 1.3447, Train Acc = 0.5752 , Test Acc = 0.6000
EPOCH 40 : Train Loss = 1.3326, Train Acc = 0.5861 , Test Acc = 0.6083
EPOCH 50 : Train Loss = 1.3273, Train Acc = 0.5623 , Test Acc = 0.5802
EPOCH 60 : Train Loss = 1.3092, Train Acc = 0.6021 , Test Acc = 0.6133
EPOCH 70 : Train Loss = 1.2992, Train Acc = 0.6008 , Test Acc = 0.6189
EPOCH 80 : Train Loss = 1.2903, Train Acc = 0.5994 , Test Acc = 0.6179
EPOCH 90 : Train Loss = 1.2939, Train Acc = 0.5974 , Test Acc = 0.6133
EPOCH 100 : Train Loss = 1.2877, Train Acc = 0.6021 , Test Acc = 0.6193
EPOCH 110 : Train Loss = 1.2905, Train Acc = 0.6030 , Test Acc = 0.6221
EPOCH 120 : Train Loss = 1.2902, Train Acc = 0.6028 , Test Acc = 0.6216
EPOCH 130 : Train Loss = 1.2948, Train Acc = 0.6028 , Tes

# GIN

In [ ]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GINGraph(
    in_channels=num_node_features,
    hidden_channels=25,
   out_channels=num_classes,
   num_layers=5
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-2)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(100):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.4145, Train Acc = 0.4736 , Test Acc = 0.4492
EPOCH 10 : Train Loss = 1.4369, Train Acc = 0.5624 , Test Acc = 0.5770
EPOCH 20 : Train Loss = 1.4345, Train Acc = 0.5530 , Test Acc = 0.5697
EPOCH 30 : Train Loss = 1.4328, Train Acc = 0.5530 , Test Acc = 0.5697
EPOCH 40 : Train Loss = 1.4319, Train Acc = 0.5462 , Test Acc = 0.5595
EPOCH 50 : Train Loss = 1.4325, Train Acc = 0.5530 , Test Acc = 0.5697
EPOCH 60 : Train Loss = 1.4327, Train Acc = 0.5538 , Test Acc = 0.5715
EPOCH 70 : Train Loss = 1.4309, Train Acc = 0.5525 , Test Acc = 0.5669
EPOCH 80 : Train Loss = 1.4298, Train Acc = 0.5493 , Test Acc = 0.5628
EPOCH 90 : Train Loss = 1.4322, Train Acc = 0.5525 , Test Acc = 0.5669
FINAL: Train Loss = 1.4342, Train Acc = 0.5478 , Test Acc = 0.5614


# GAT

In [5]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATGraph(
    in_channels=num_node_features,
    hidden_channels=32,
    out_channels=num_classes,
    heads = 2,
    layers= 3
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(150):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
    scheduler.step(train_loss)
    
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.9413, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 10 : Train Loss = 1.5982, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 20 : Train Loss = 1.5781, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 30 : Train Loss = 1.5705, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 40 : Train Loss = 1.5637, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 50 : Train Loss = 1.5573, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 60 : Train Loss = 1.5593, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 70 : Train Loss = 1.5533, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 80 : Train Loss = 1.5509, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 90 : Train Loss = 1.5474, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 100 : Train Loss = 1.5483, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 110 : Train Loss = 1.5488, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 120 : Train Loss = 1.5425, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 130 : Train Loss = 1.5435, Train Acc = 0.5105 , Tes